# Yeşil toplu hesapların analizi

Gerekli Python kütüphanelerini kuralım

In [ ]:
%pip install rekognize  # Twitter verilerini çekmek icin
%pip install sklearn    # Profillerde geçen ifadelerin analizi için

Twitter istemcimizi oluşturalım. Bunun için https://rekognize.io/twitter/ adresinden token almanız gerekiyor. 

In [ ]:
from rekognize.twitter import UserClient

ACCESS_TOKEN = 'ACCESS_TOKEN'                # <- kendi token'ın
ACCESS_TOKEN_SECRET = 'ACCESS_TOKEN_SECRET'  # <- kendi token secret'ın

client = UserClient(
    ACCESS_TOKEN,
    ACCESS_TOKEN_SECRET
)

## Veri çekme
Değişkenlerimizi oluşturalım.

In [ ]:
users = {}
max_id = None

Bir döngü içerisinde paylaşımları çekelim. 

In [10]:
while True:
    r = client.api.search.tweets.get(
        q='🟢',  # <- burada kutu olarak görünen yeşil nokta
        lang='tr',
        count=100,
        include_entities=False,
        max_id=max_id,
    )
    users.update({
        s.user.id: s.user for s in r.data.statuses
    })
    if not r.data.statuses or r.remaining == 0:
        break
    max_id = r.data.statuses[-1].id - 1

Şimdi ilgili kullanıcılar `users` sözlük değişkeninde. 

## Profil tanımlarının incelenmesi

Profil tanımlarında en cok geçen kelimeleri `sklearn` kütüphanesinin `CountVectorizer`ı ile tespit ediyoruz. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

c_vec = CountVectorizer(ngram_range=(1, 4))  # 1 - 4 kelimelik ifadeler
ngrams = c_vec.fit_transform(
    [user.description for user in users.values()]
)

vocab = c_vec.vocabulary_

count_values = ngrams.toarray().sum(axis=0)

# ifadeleri frekanslarına göre sıralayarak yazdıralım
for ng_count, ng_text in sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True):
    print(ng_count, ng_text)

## Hesapların oluşturulma zamanları

Bir döngü ile, hangi tarihte kaç hesap açılğını alalım.

In [46]:
from collections import defaultdict, Counter
from delorean import parse

timeline = Counter()

for user in users.values():
    created_at = parse(user['created_at']).datetime
    timeline.update([created_at.strftime('%Y-%m-%d')])

for day, c in timeline.items():
    print(day, c)

Artık `timeline` değişkeninde tarihlere göre açılan hesap sayıları var. 

Elde ettiğimiz verileri artık istediğimiz gibi analiz edebilir ve görselleştirebiliriz. 